In [1]:
using FermiCG
using Printf
using Test
using JLD2 

@load "../test/_testdata_cmf_h12_64bit.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);

v_bst = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases)
v_tps = FermiCG.TPSCIstate(clusters, FockConfig(init_fspace))
FermiCG.eye!(v_tps)

s_bst = FermiCG.build_compressed_1st_order_state(v_bst, cluster_ops, clustered_ham, nbody=4, thresh=1e-24)
s_tps = FermiCG.open_matvec_thread(v_tps, cluster_ops, clustered_ham, thresh=1e-32)

@show size(s_bst);
@show size(s_tps);





 Remove duplicates


 Number of terms reduced from  2525 to  1105
IDX001:DIM0016:

001|002|


IDX002:DIM0016:003|004|


IDX003:DIM0256:005|006|007|008|


...

 In open_matvec_thread
 Setup threaded jobs:                                0.000671 seconds (1.58 k allocations: 145.469 KiB)
 Number of jobs:    525
 Number of threads: 4


 Compute matrix-vector:                            

  3.388449 seconds (15.24 M allocations: 732.094 MiB, 3.71% gc time, 99.80% compilation time)


 Now collect thread results :                      

  0.013009 seconds (1.96 k allocations: 113.500 KiB, 96.61% compilation time)
size(s_bst) = (16298, 1)
size(s_tps) = (49378, 1)


# BST PT2

In [43]:
e_var = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);
e_pt = FermiCG.compute_pt2_energy(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-32, max_iter=100, tol=1e-12, verbose=0);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                

  0.001601 seconds (4.08 k allocations: 30.747 MiB)


 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 


 Diagonalization time:                               0.000946 seconds
 Compute <S^2>:                                    

  0.000389 seconds (1.35 k allocations: 399.688 KiB)


...

  0.000346 seconds (1.35 k allocations: 399.688 KiB)


  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Diagonalization time:                               0.004564 seconds
 Compute <0|H0|0>:                                 

  0.000311 seconds (665 allocations: 363.547 KiB)
 Number of jobs:    525
 Number of threads: 4


----------------------------------------------------------------------------------------------------

 Time spent computing E2:                               0.4 s Allocated:    6.7e-01 GB
 State   1: E(PT2) corr:                          -0.00992596
 State   1: E(PT2):                              -18.32703491
 ==================================================================|


In [47]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-32)
    
# FermiCG.project_out!(s_bst, v, thresh=1e-32)
# @show size(s_bst)
# s_bst = FermiCG.compress(s_bst, thresh=1e-32)
# @show size(s_bst)
v_pt, e_pt = FermiCG.hylleraas_compressed_mp2a(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                

  0.001329 seconds (4.21 k allocations: 30.765 MiB)


 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 


 Diagonalization time:                               0.000788 seconds
 Compute <S^2>:                                    

  0.000338 seconds (1.35 k allocations: 399.688 KiB)


...

  Root      <0|H|0>      <0|F|0>
     1 -18.31710895 -10.81361409


 Compute <X|F|0>:                                    0.000518 seconds   9.80e-04 Gb


 Cache zeroth-order Hamiltonian:                     0.028012 seconds   4.19e-02 Gb


 Time to solve for PT1 with conjugate gradient:      0.476703 seconds   8.39e-01 Gb


 Compute <0|H|1>:                                    0.026666 seconds   1.67e-02 Gb
 State   1: E(PT2) corr:                           0.01193465
 State   1: E(PT2):                              -18.30517430


In [49]:
e_pt, v_pt = FermiCG.do_fois_pt2(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-64, max_iter=50, tol=1e-8);

 |== Solve for BST PT1 Wavefunction ================================
 H0          : Hcmf
 max_iter    : 50
 nbody       : 4
 thresh_foi  : 1.0e-64
 tol         : 1.0e-8
 opt_ref     : true
 verbose     : true

 Length of Reference:                                       1
 Solve zeroth-order problem:                       
 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                

  0.033188 seconds (4.08 k allocations: 30.747 MiB, 95.51% gc time)


 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 


 Diagonalization time:                               0.000873 seconds
 Compute <S^2>:                                    

  0.000365 seconds (1.35 k allocations: 399.688 KiB)


...

 Compute <0|H0|0>:                                 

  0.000185 seconds (510 allocations: 348.141 KiB)
  Root      <0|H|0>      <0|F|0>
     1 -18.31710895 -10.81361409
 Compute <X|F|0>:                                  

  0.000456 seconds (7.96 k allocations: 935.406 KiB)
 Cache zeroth-order Hamiltonian:                     0.009079 seconds (70.86 k allocations: 38.991 MiB)


 Time to solve for PT1 with conjugate gradient:      0.367628 seconds
 Compute <0|H|1>:                                  

  0.025978 seconds (252.27 k allocations: 15.367 MiB)
 State   1: E(PT2) corr:                          -0.00992618
 State   1: E(PT2):                              -18.32703513
 ==================================================================|


In [112]:
ints.h0

11.795261590783612

# TPSCI PT2

In [18]:
function pt1(ψ, cluster_ops, clustered_ham)
    σ = FermiCG.build_compressed_1st_order_state(ψ, cluster_ops, clustered_ham, nbody=4, thresh=1e-6)
    
end
pt1(ψ, cluster_ops, clustered_ham)

 Number of tasks:                                         661
 Compute tasks:                                      0.077710 seconds (597.90 k allocations: 56.236 MiB)
 Add results together:                             

  0.016173 seconds (85.70 k allocations: 9.271 MiB)



 --------------------------------------------------
 ---------- # Fockspaces -------------------:   485  
 ---------- # Configs    -------------------:  5500  
 ---------- Root ---------------------------:     1  
 --------------------------------------------------
 Printing contributions greater than: 0.010000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.027                                   ( 1,1 )( 2,1 )( 1,2 )( 1,1 )( 1,1 )
     0.027           16        48            1:1    1:2   1:24    1:1    1:1
 ---------                               ----------          
 0.028                                   ( 1,1 )( 0,1 )( 3,2 )( 1,1 )( 1,1 )
     0.027           16        48            1:1    1:2   1:24    1:1    1:1
 ---------                               ----------          
 0.027                                   ( 2,1 )( 0,1 )( 2,2 )( 1,1 )( 1,1 )
     0.026           4         4             1:2    1